# PRACTICA 3

**Nombre:** Jorge Osvaldo Gonzalez Gonzalez
**e-mail:** jorge.gonzalez1611@alumnos.udg.mx

## MODULES

In [1]:
import math
import os 

import numpy as np
import pandas as pd
import panel as pn
import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance


## CLASES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

### Provided

In [3]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

### Own

#### Parsers

In [4]:
def np_arrays_to_df(arrays, column_names): 
    df_res = pd.DataFrame(columns=column_names)
    for i in range(0, len(arrays[0])):
        df_res = pd.concat([df_res, pd.DataFrame(columns=column_names, data=[[arrays[j][i]  for j in range(0, len(column_names))]])], ignore_index=True)
    return df_res

#### Files

In [5]:
def save_df_to_csv(df_in, path, name):
    os.makedirs(path, exist_ok=True)
    df_in.to_csv(f'{path}/{name}.csv', index=False)

#### Generators

###### Brownian Motion

##### Correlated Random Walk

In [6]:
def generate_crw(steps, speed, coef):
    cur_pos = Vec2d(speed, 0)
    crw_df = pd.DataFrame([{'x': 0, 'y': 0}])
    crw_turns = wrapcauchy.rvs(c=coef, loc=0, size=steps-1)
    for i in range(1, steps):
        cur_pos = cur_pos.rotated(crw_turns[i-1])
        crw_df = pd.concat([crw_df, pd.DataFrame([{'x': crw_df.iloc[i-1, 0] + cur_pos.x, 'y': crw_df.iloc[i-1, 1] + cur_pos.y}])]) 
    return crw_df

##### Levy Flight

In [7]:
def generate_levy(steps, speed, cauchy, alpha, beta, std_steps):
    cur_pos_lw = Vec2d(speed, 0)
    lw_2d_df = pd.DataFrame([{"x": 0, "y": 0}])
    lw_displacements = levy_stable.rvs(alpha=alpha, beta=beta, loc=std_steps)
    # Generate trajectory by accumulation of displacements4
    i = 1
    while i <= steps:
        turn_angle = wrapcauchy.rvs(c=cauchy, loc=0)
        cur_pos_lw = cur_pos_lw.rotated(turn_angle)
        cur_steps = abs(round(levy_stable.rvs(alpha=alpha, beta=beta, loc=std_steps))) 

        for j in range(0, int(cur_steps)):
            lw_2d_df = pd.concat([lw_2d_df, pd.DataFrame([{'x': lw_2d_df.x[i-1]+cur_pos_lw.x, 'y': lw_2d_df.y[i-1]+cur_pos_lw.y}])], ignore_index=True)
            i+=1
            if i > steps:
                break
    return lw_2d_df

#### METRICS

###### MSD

In [8]:
###################################################################################
# MSD VECTOR
###################################################################################
def MSDVector(df_in):
    displacement_vector = np.zeros(df_in.shape[0] - 1)
    for tau in range(1,df_in.shape[0]):
        ## start - Add your code here
        MSD = np.array([distance.euclidean(df_in.iloc[i-tau], df_in.iloc[i]) ** 2
            for i in range(tau, df_in.shape[0], 1)])
        displacement_vector[tau-1] = MSD.mean()
    return displacement_vector